In [41]:
import pandas as pd

# Load clustered data
df_clusters = pd.read_csv(r"C:\Users\Administrator\Desktop\Python\ML\ProjectPlan\KNN-SubClusters\knn_subclusters.csv")

# Load vehicle master
df_vehicles = pd.read_csv(r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\vehicle_type_map_with_capacity.csv")

# Add tracking columns
df_vehicles["Remaining_Weight"] = df_vehicles["Weight_Capacity (kg)"]
df_vehicles["Remaining_Volume"] = df_vehicles["Volume_Capacity (L)"]
df_vehicles["Used_By_Orders"] = [[] for _ in range(len(df_vehicles))]

# Init cluster predictions
df_clusters["Predicted_Vehicles"] = ""


In [42]:
def assign_order_to_vehicles(row, vehicle_pool):
    order_id = row["Product_ID"]
    rem_weight = row["Total_Weight"]
    rem_volume = row["Total_Volume"]
    assigned_vehicles = []

    for v_idx, vehicle in vehicle_pool.iterrows():
        if rem_weight <= 0 and rem_volume <= 0:
            break

        v_weight = vehicle["Remaining_Weight"]
        v_volume = vehicle["Remaining_Volume"]

        if v_weight <= 0 or v_volume <= 0:
            continue

        wt_used = min(rem_weight, v_weight)
        vol_used = min(rem_volume, v_volume)

        # Update remaining
        vehicle_pool.at[v_idx, "Remaining_Weight"] -= wt_used
        vehicle_pool.at[v_idx, "Remaining_Volume"] -= vol_used
        vehicle_pool.at[v_idx, "Used_By_Orders"].append(order_id)

        assigned_vehicles.append(vehicle["Vehicle_ID"])
        rem_weight -= wt_used
        rem_volume -= vol_used

    if rem_weight > 0 or rem_volume > 0:
        return "UNASSIGNABLE"

    return " | ".join(assigned_vehicles)


In [43]:
for i, row in df_clusters.iterrows():
    is_sensitive = row["Fragile_Flag"] == 1 or row["Temp_Sensitive_Flag"] == 1

    # Filter pool based on type
    pool = df_vehicles[df_vehicles["Vehicle_Type"] == ("Specialised" if is_sensitive else "General")].copy()
    pool = pool.reset_index(drop=True)

    # Assign using greedy logic
    assigned = assign_order_to_vehicles(row, df_vehicles[df_vehicles["Vehicle_Type"] == pool["Vehicle_Type"][0]])
    df_clusters.at[i, "Predicted_Vehicles"] = assigned


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10208\2028344666.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '8977.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  vehicle_pool.at[v_idx, "Remaining_Weight"] -= wt_used
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10208\2028344666.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '34990.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  vehicle_pool.at[v_idx, "Remaining_Volume"] -= vol_used
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10208\2028344666.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '14977.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  vehicle_pool.at

In [44]:
# Save the output with all merged info + predictions
output_path = r"final_vehicle_assignment.csv"
# df_clusters.to_csv(output_path, index=False)
# print(f"✅ Done: Vehicle assignments saved to {output_path}")

df_clusters.to_csv(output_path, index=False)
print("✅ Final merged file saved.")




✅ Final merged file saved.
